In [ ]:
import numpy as np
import pandas as pd
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
df=pd.read_csv('./dataset_transoform/PPG/ppg_01.csv',header=None)

def nn_intervals_from_signal(signal,sampling_freq:int):
    dt = 1/sampling_freq # Sampling interval
    peaks, _ = find_peaks(signal, height=0.5*np.max(signal),distance=100)
    peaks_diff=peaks*dt
    nn_intervals = np.diff(peaks_diff)
    return nn_intervals


In [ ]:
import numpy as np
from scipy import signal 
import matplotlib.pyplot as plt
from scipy.stats import skew,kurtosis,entropy,median_abs_deviation

def onecycle(ppg):
        b, a = signal.butter(2, 0.5, 'highpass')
        ppg1 = signal.filtfilt(b, a, ppg)

        # Normalize the data to have a mean of 0 and standard deviation of 1
        ppg1 = (ppg1 - np.mean(ppg1)) / np.std(ppg1)

        rpeaks, = ecg.hamilton_segmenter(signal=ppg1, sampling_rate=128)
        rr_intervals = np.diff(rpeaks)

        # Compute the time-domain HRV features
        hrv_mean = np.mean(rr_intervals)
        hrv_sdnn = np.std(rr_intervals)
        hrv_rmssd = np.sqrt(np.mean(np.square(np.diff(rr_intervals))))

        length=len(ppg)
        peaks, _ = find_peaks(ppg, distance=15)
        min1=9999999
        min2=9999999
        for i in range(peaks[0],peaks[1]):
            if ppg[i]<min1:
                min1=i
        for i in range(peaks[1],peaks[2]):
            if ppg[i]<min2:
                min2=i
        if abs(peaks[2]-peaks[1])>22.5:
            min2=peaks[0]+min1
    #     print(min1,min2)
        final=ppg[min1-1:min2]
        final=ppg
        mean=np.mean(final)
        std=np.std(final)
        median = np.median(final)
        skewness=skew(final)
        kurt=kurtosis(final)
        p10,p25,p30,p50,p75,p80,p90 = np.nanpercentile(final,[10,25,30,50,75,80,90])
        IQR = p75-p25
        EN = entropy(final)
        mid_AD = median_abs_deviation(final)
        mean_AD = sum([abs(i-np.mean(final)) for i in final])/len(final)
        RMS =  np.sqrt(sum([i**2 for i in final])/len(final))
        spec_EN = entropy(final)/np.log2(len(final))

        ppg_diff = np.diff(ppg)
        # calculate the second derivative of the PPG signal
        ppg_diff2 = np.diff(ppg_diff)
        # calculate the activity, mobility, and complexity of the PPG signal
        activity = np.var(ppg)
        mobility = np.sqrt(np.var(ppg_diff) / activity)
        complexity = np.sqrt(np.var(ppg_diff2) / np.var(ppg_diff))

        env = np.abs(signal.hilbert(ppg))
        # calculate the mean and standard deviation of the envelope
        env_mean = np.mean(env)
        env_std = np.std(env)
        # calculate the mean and standard deviation of the amplitude modulation
        am = ppg / env
        am_mean = np.mean(am)
        am_std = np.std(am)
        #print([mean,std,skewness,kurt,p10,p25,p30,p50,p75,p80,p90])
        return [hrv_rmssd,hrv_sdnn, hrv_mean,mean,median,std,kurt,skewness,p10,p25,p30,p50,p75,p80,p90,IQR,EN,mid_AD,mean_AD,RMS,spec_EN,activity,mobility,complexity,env_mean,env_std,am_mean,am_std]

def gsr_features(input_signal):
    # Compute skin conductance level (SCL)
    scl = np.mean(input_signal)
    # Compute skin conductance response (SCR)
    scr = np.max(input_signal) - np.min(input_signal)
    # Compute rise time and half-recovery time of SCR
    peaks, _ = signal.find_peaks(input_signal, prominence=0.1)
    rise_time = peaks[0] - np.argmin(input_signal[:peaks[0]])
    half_recovery_time = peaks[0] - np.argmin(input_signal[peaks[0]:]) + np.argmax(input_signal[peaks[0]:])
    # Compute peak amplitude and area under the curve (AUC) of SCR
    peak_amp = np.max(input_signal) - np.min(input_signal[:peaks[0]])
    auc = np.trapz(input_signal[:peaks[0]])
    # Compute number of SCRs
    n_scr = len(peaks)
    # Compute slope and intercept of tonic GSR
    x = np.arange(len(input_signal))
    slope, intercept = np.polyfit(x, input_signal, 1)
    # Compute latency and duration of the GSR response
    if n_scr > 0:
        latencies = peaks - np.argmin(input_signal[:peaks[0]])
        durations = np.diff(peaks)
        mean_latency = np.mean(latencies)
        mean_duration = np.mean(durations)
    else:
        mean_latency = 0
        mean_duration = 0

    return [scl,scr,rise_time,half_recovery_time,peak_amp,auc,n_scr,slope,intercept,mean_latency,mean_duration]


def nn_intervals_time_domain_features(input_signal):
    meanNN = np.mean(input_signal)
    medianNN = np.median(input_signal)
    sdnn = np.std(input_signal)
    nn50 = [x>0.05 for x in np.diff(input_signal)].count(True)
    pNN50 = nn50 / len(input_signal) * 100
    rmssd = np.sqrt(np.mean(np.square(np.diff(input_signal))))
    return [meanNN,medianNN,nn50,pNN50,rmssd,sdnn]

def freq_domain_features_from_signal(input):
    """ 
    params : input_signal -> np array
    params: fs -> int

    VLFa area of VLF (0-0.04hz)
    LFa area of VLF   (0.0.4-0.15hz)
    HFa area of VLF   (0.15-0.4hz)
    TOTALa total area of all band
    VLFh highest power in VLF
    LFh highest power in LF
    HFh highest power in HF
    VLFp VLFa/TOTALa
    LFp LFa/TOTALa
    HFp HFa/TOTALa
    LFn normalized area of LF LFa/(LFa+HFa)
    HFn normalized area of HF HFa/(LFa+HFa)
    LFHF ratio of LFa and HFa

    return : [lf_area,hf_area,vlf_p,lf_p,hf_p,lf_n,lfhf]
    """
    # f = np.linspace(0, fs/2, len(input_signal)//2+1)  # Frequency range (0 to Nyquist)
    # # Apply a Hamming window to the input_signal
    # window = signal.hamming(len(input_signal))
    # input_signal_windowed = input_signal * window

    # # Compute the FFT of the windowed input_signal
    # fft = np.fft.rfft(input_signal_windowed)

    # Compute the power spectral density (PSD)
    input_signal=signal.detrend(input, type='constant')
    f, psd =signal.welch(input_signal, nperseg=1024,window='hamming')

    # Define the frequency bands
    vlf_band = (0.003, 0.04)  # Very low frequency (0.003-0.04 Hz)
    lf_band = (0.04, 0.15)    # Low frequency (0.04-0.15 Hz)
    hf_band = (0.15, 0.4)     # High frequency (0.15-0.4 Hz)

    # Compute the indices corresponding to the frequency bands
    vlf_idx = np.where(np.logical_and(f >= vlf_band[0], f <= vlf_band[1]))[0]
    lf_idx = np.where(np.logical_and(f >= lf_band[0], f <= lf_band[1]))[0]
    hf_idx = np.where(np.logical_and(f >= hf_band[0], f <= hf_band[1]))[0]
    # Compute the area under the PSD curve for each band
    vlf_area = np.trapz(psd[vlf_idx], f[vlf_idx])
    lf_area = np.trapz(psd[lf_idx], f[lf_idx])
    hf_area = np.trapz(psd[hf_idx], f[hf_idx])
    total_area = np.trapz(psd, f)
    # Find the frequency at which the PSD has the highest value for each band
    vlf_h = f[vlf_idx[np.argmax(psd[vlf_idx])]]
    lf_h = f[lf_idx[np.argmax(psd[lf_idx])]]
    hf_h = f[hf_idx[np.argmax(psd[hf_idx])]]

    max_power=np.max(psd)

    # Compute the VLFp, LFp, and HFp features
    vlf_p = vlf_area / total_area
    lf_p = lf_area / total_area
    hf_p = hf_area / total_area

    # Compute the LFn and HFn features
    lf_n = lf_area / (lf_area + hf_area)
    hf_n = hf_area / (lf_area + hf_area)

    # Compute the LF/HF ratio (LFHF)
    lfhf = lf_area / hf_area
    #  VLFa, LFa,
    # HFa, TOTALa, VLFp, LFp, HFp
    ppg_freqs, ppg_psd = signal.welch(input, fs=128, nperseg=256)
    alpha = np.mean(ppg_psd[(ppg_freqs >= 8) & (ppg_freqs <= 13)])
    beta = np.mean(ppg_psd[(ppg_freqs >= 13) & (ppg_freqs <= 30)])
    gamma = np.mean(ppg_psd[(ppg_freqs >= 30) & (ppg_freqs <= 100)])
    theta = np.mean(ppg_psd[(ppg_freqs >= 4) & (ppg_freqs <= 8)])
    delta = np.mean(ppg_psd[(ppg_freqs >= 0.5) & (ppg_freqs <= 4)])
    return [vlf_area,lf_area,hf_area,total_area,vlf_p,lf_p,hf_p,vlf_h,lf_h,hf_h,lf_n,hf_n,lfhf,max_power,alpha,beta,gamma,theta,delta]

def extract_features(input):
    return freq_domain_features_from_signal(input)+nn_intervals_time_domain_features(input)+gsr_features(input)+onecycle(input)

In [ ]:
data=np.genfromtxt('dataset_transoform/GSR/gsr_01.csv',delimiter=',')


In [ ]:
extract_features(data[0])

In [ ]:
import pandas as pd
from scipy.stats import skew,kurtosis
from ast import literal_eval
import pandas as pd
import matplotlib.pyplot as plt 
from scipy import signal
from scipy.fftpack import rfft, irfft, fftfreq, fft,ifft
from scipy.signal import find_peaks, peak_prominences
from numpy import trapz
from scipy.stats import skew as find_skew
import numpy as np
from scipy.stats import skew

def motion_removal(data):
    
    Fs = 100;                                                   #Sampling Frequency (Hz)
    Fn = Fs/2;                                                  # Nyquist Frequency (Hz)
    Ws = 0.5/Fn;                                                # Passband Frequency Vector (Normalised)
    Wp = 1.5/Fn;                                                # Stopband Frequency Vector (Normalised)
    Rp =   1;                                                   # Passband Ripple (dB)
    Rs = 150; 
    N, Wn = signal.ellipord(Wp,Ws,Rp,Rs)
    z,p,k = signal.ellip(N,Rp,Rs,Wn,'high',output='zpk')
    sos= signal.zpk2sos(z, p, k)
    y = signal.sosfiltfilt(sos, data)
    return y
def plotting(ir):
    plt.figure(figsize=(16,4))
    plt.plot(ir,label='IR')
    plt.legend()
    
def normalize_values(read_file,write_file):
    # Load CSV file
    df = pd.read_csv(read_file, header=None,delimiter=',')

    # # Transpose dataframe to treat first row as values
    df = df.transpose()

    # Normalize values using min-max scaling
    df = (df - df.min()) / (df.max() - df.min())

    # # Transpose dataframe back to original format
    df = df.transpose()

    # Save normalized values to CSV file
    df.to_csv(write_file, header=None, index=None)

def data_builder(input_signal,val,arr):
    if(val>=5): val_class=1
    else: val_class=0
    if(arr>=5): arr_class=1
    else: arr_class=0
    return extract_features(input_signal)+[val,arr]



In [ ]:
import os

infiles=os.listdir('dataset_transoform/GSR/')
tarfiles=os.listdir('dataset/labels/')

data_ext=np.empty((800,66))
ind=0
for file in range(len(infiles)):
    data=np.genfromtxt('dataset_transoform/GSR/'+infiles[file],delimiter=',')
    target=np.genfromtxt('dataset/labels/'+tarfiles[file],delimiter=',')
    print(data.shape)
    print(target.shape)
    for i in range(len(data)):
        stats_feature=data_builder(data[i],target[i][0],target[i][1])
        data_ext[ind]=np.array(stats_feature)
        ind+=1


In [ ]:
import pandas as pd

df=pd.DataFrame(data_ext,index=None,columns=["vlf_area","lf_area","hf_area","total_area","vlf_p","lf_p","hf_p","vlf_h","lf_h","hf_h","lf_n","hf_n","lfhf","max_power","alpha","beta","gamma","theta","delta","meanNN","medianNN","nn50","pNN50","rmssd","sdnn","scl","scr","rise_time","half_recovery_time","peak_amp","auc","n_scr","slope","intercept","mean_latency","mean_duration","hrv_rmssd","hrv_sdnn"," hrv_mean","mean","median","std","kurt","skewness","p10","p25","p30","p50","p75","p80","p90","IQR","EN","mid_AD","mean_AD","RMS","spec_EN","activity","mobility","complexity","env_mean","env_std","am_mean","am_std","val","arr"])

In [ ]:
df.to_csv('gsr_features_64.csv',index=None)

In [160]:
## importing lazypredict library
import lazypredict
### importing LazyClassifier for classification problem
from lazypredict.Supervised import LazyClassifier
### importing LazyClassifier for classification problem because here we are solving Classification use case.
from lazypredict.Supervised import LazyClassifier
### importing breast Cancer Dataset from sklearn
from sklearn.datasets import load_breast_cancer
### spliting dataset into training and testing part
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,:-2].to_numpy(),df.iloc[:,-1:].to_numpy().reshape(800), test_size=.5, random_state =123)
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric = None)
### fitting data in LazyClassifier
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
### lets check which model did better on Breast Cancer Dataset
print(models)

100%|██████████| 29/29 [00:00<00:00, 108.15it/s]

Empty DataFrame
Columns: [Accuracy, Balanced Accuracy, ROC AUC, F1 Score, Time Taken]
Index: []


In [162]:
df.to_numpy()[0]

array([ 1.24169325e-03,  2.91328920e-04,  1.57334995e-03,  3.42971469e-03,
        3.62039809e-01,  8.49426108e-02,  4.58740766e-01,  1.85546875e-02,
        1.22070312e-01,  2.49023438e-01,  1.56235439e-01,  8.43764561e-01,
        1.85164732e-01,  2.61167683e-01,  1.56184382e-05,  8.49751572e-06,
        5.67859791e-05,  5.07894209e-05,  2.89817429e-04,  5.02835148e-01,
        5.02603862e-01,  1.45700000e+03,  1.80679563e+01,  6.83624946e-02,
        5.90566264e-02,  5.02835148e-01,  1.00000000e+00,  1.30000000e+01,
        1.16000000e+02,  5.72942942e-01,  6.04435564e+01,  5.78000000e+02,
       -2.15693163e-08,  5.02922105e-01,  4.53119723e+03,  1.36759099e+01,
        7.92847373e+01,  5.48350028e+01,  6.88620690e+01,  5.02835148e-01,
        5.02603862e-01,  5.90566264e-02,  6.71997836e+00, -1.10255040e-01,
        4.42053718e-01,  4.74154783e-01,  4.80393732e-01,  5.02603862e-01,
        5.32281166e-01,  5.39856897e-01,  5.63330842e-01,  5.81263828e-02,
        8.98802709e+00,  